In [1]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import datetime
import math

In [2]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
pricedata = pd.read_csv(datafile, names=colnames)
pricedata = pricedata[pricedata['Date'] == '02/12/2019'].copy()

pricedata['DateTime'] = (pricedata['Date']+pricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del pricedata['Date']
del pricedata['Time']

pricedata = pricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in pricedata['DateTime']]].copy()

In [3]:
pricedata

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskLow,AskHigh,AskClose,DateTime
765481,110.46,110.46,110.42,110.42,110.51,110.51,110.49,110.49,2019-02-12 09:30:00
765482,110.48,110.53,110.44,110.47,110.49,110.54,110.48,110.51,2019-02-12 09:31:00
765483,110.48,110.51,110.48,110.48,110.49,110.54,110.49,110.50,2019-02-12 09:32:00
765484,110.46,110.53,110.46,110.53,110.50,110.54,110.50,110.54,2019-02-12 09:33:00
765485,110.50,110.53,110.50,110.52,110.53,110.54,110.53,110.54,2019-02-12 09:34:00
765486,110.53,110.54,110.53,110.54,110.54,110.55,110.54,110.54,2019-02-12 09:35:00
765487,110.55,110.57,110.53,110.54,110.56,110.59,110.54,110.56,2019-02-12 09:36:00
765488,110.56,110.61,110.55,110.59,110.56,110.63,110.56,110.60,2019-02-12 09:37:00
765489,110.60,110.66,110.60,110.66,110.61,110.68,110.61,110.68,2019-02-12 09:38:00
765490,110.66,110.68,110.66,110.68,110.68,110.69,110.68,110.69,2019-02-12 09:39:00


In [4]:
def candlestickChart():
    
    data = [ dict(
            type = 'candlestick',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            low = pricedata['BidLow'],
            open = pricedata['BidOpen'],
            close = pricedata['BidClose'],
            high = pricedata['BidHigh']),
            
#             dict(
#             type = 'scatter',
#             x = [40,80],
#             y = [closedata[40], closedata[80]],
#             mode = 'markers',
#             marker = dict(
#                 size = 10,
#                 opacity = 0.8,
#                 reversescale = True,
#                 autocolorscale = False,
#                 symbol = 'circle',
#                 line = dict(
#                     width=1,
#                     color='black'
#                 ),
#                 color='blue')
#             )
           ]

    layout = dict(
            title = 'Candlestick Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

In [5]:
candle = candlestickChart()
candle

FigureWidget({
    'data': [{'close': array([110.42, 110.47, 110.48, ..., 111.08, 111.11, 111.11]),
          …